In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/train_germination.npy')
val_x = np.load('/home/ckwwk/data/germination_data/val_germination.npy')
test_x = np.load('/home/ckwwk/data/germination_data/test_germination.npy')

In [4]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/label/germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [5]:
from keras.utils import to_categorical
train_y = label_df[0:1000]
train_y = to_categorical(train_y)

In [6]:
val_y = label_df[1000:1250]
val_y = to_categorical(val_y)
test_y = label_df[1250:1536]
test_y = to_categorical(test_y)

In [7]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [8]:
input_layer = Input((61, 61, 275, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 9), activation='relu',padding='same')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 9), activation='relu',padding='same')(conv_layer1)

## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 6))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 9), activation='relu',padding='same')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 9), activation='relu',padding='same')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 6))(conv_layer4)

conv_layer5 = Conv3D(filters=128, kernel_size=(3, 3, 9), activation='relu',padding='same')(pooling_layer2)
conv_layer6 = Conv3D(filters=256, kernel_size=(3, 3, 9), activation='relu',padding='same')(conv_layer5)
pooling_layer3 = MaxPool3D(pool_size=(2, 2, 6))(conv_layer4)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer3 = BatchNormalization()(pooling_layer3)
flatten_layer = Flatten()(pooling_layer3)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

W0805 20:00:42.866153 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 20:00:42.867767 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 20:00:42.872012 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 20:00:43.046862 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pleas

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 275, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 61, 61, 275, 8)    656       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 61, 61, 275, 16)   10384     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 30, 45, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 30, 30, 45, 32)    41504     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 30, 30, 45, 64)    165952    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 15, 15, 7, 64)     0         
__________

In [10]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model,gpus=4)

W0805 20:00:43.161859 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [11]:
adam = Adam(lr=0.005, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W0805 20:00:44.753219 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
filepath = "best-model(V2).hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='/home/ckwwk/tensor_log(V2)',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [13]:
history = model.fit(x=train_x, y=train_y, batch_size=32, epochs=1000, validation_data=(val_x,val_y),callbacks=callbacks_list)

W0805 20:00:47.247823 140134239274816 deprecation.py:323] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 250 samples


W0805 20:00:57.247061 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0805 20:00:57.278651 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0805 20:00:57.281128 140134239274816 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/1000
1000/1000 [==============================] - 51s 51ms/step - loss: 5.8114 - acc: 0.6220 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00001: acc improved from -inf to 0.62200, saving model to best-model(V2).hdf5
Epoch 2/1000
1000/1000 [==============================] - 34s 34ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00002: acc improved from 0.62200 to 0.63200, saving model to best-model(V2).hdf5
Epoch 3/1000
1000/1000 [==============================] - 36s 36ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00003: acc did not improve from 0.63200
Epoch 4/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00004: acc did not improve from 0.63200
Epoch 5/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00005: acc did not improve from 0.63


Epoch 00044: acc did not improve from 0.63200
Epoch 45/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00045: acc did not improve from 0.63200
Epoch 46/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00046: acc did not improve from 0.63200
Epoch 47/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00047: acc did not improve from 0.63200
Epoch 48/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00048: acc did not improve from 0.63200
Epoch 49/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00049: acc did not improve from 0.63200
Epoch 50/1000
1000/1000

...

Epoch 353/1000
1000/1000 [==============================] - 34s 34ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00353: acc did not improve from 0.63200
Epoch 354/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00354: acc did not improve from 0.63200
Epoch 355/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00355: acc did not improve from 0.63200
Epoch 356/1000
1000/1000 [==============================] - 36s 36ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00356: acc did not improve from 0.63200
Epoch 357/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00357: acc did not improve from 0.63200
Epoch 358/1000
1000/1000 [==============================] - 35s 3

Epoch 397/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00397: acc did not improve from 0.63200
Epoch 398/1000
1000/1000 [==============================] - 34s 34ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00398: acc did not improve from 0.63200
Epoch 399/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00399: acc did not improve from 0.63200
Epoch 400/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00400: acc did not improve from 0.63200
Epoch 401/1000
1000/1000 [==============================] - 40s 40ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00401: acc did not improve from 0.63200
Epoch 402/1000
1000/1000 [==============================] - 39s 3

Epoch 441/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00441: acc did not improve from 0.63200
Epoch 442/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00442: acc did not improve from 0.63200
Epoch 443/1000
1000/1000 [==============================] - 34s 34ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00443: acc did not improve from 0.63200
Epoch 444/1000
1000/1000 [==============================] - 35s 35ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00444: acc did not improve from 0.63200
Epoch 445/1000
1000/1000 [==============================] - 34s 34ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00445: acc did not improve from 0.63200
Epoch 446/1000
1000/1000 [==============================] - 34s 3

Epoch 485/1000
1000/1000 [==============================] - 40s 40ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00485: acc did not improve from 0.63200
Epoch 486/1000
1000/1000 [==============================] - 41s 41ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00486: acc did not improve from 0.63200
Epoch 487/1000
1000/1000 [==============================] - 42s 42ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00487: acc did not improve from 0.63200
Epoch 488/1000
1000/1000 [==============================] - 54s 54ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00488: acc did not improve from 0.63200
Epoch 489/1000
1000/1000 [==============================] - 38s 38ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00489: acc did not improve from 0.63200
Epoch 490/1000
1000/1000 [==============================] - 34s 3

KeyboardInterrupt: 

In [ ]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
#model.predict(test_x,batch_size=10)
model.evaluate(test_x,test_y,batch_size=32)